<a href="https://colab.research.google.com/github/Nehasatheesh04/Deep_Learning/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GAN_MNIST

In [ ]:
pip install torch torchvision matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [7]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
latent_dim = 100
img_shape = 28 * 28
batch_size = 128
epochs = 50
lr = 0.0002

# Transform for MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]
])

# Dataset
train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, img_shape),
            nn.Tanh()  # Output values between -1 and 1
        )

    def forward(self, z):
        img = self.model(z)
        return img

# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(img_shape, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()  # Output between 0 and 1
        )

    def forward(self, img):
        validity = self.model(img)
        return validity

# Initialize models
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Loss
adversarial_loss = nn.BCELoss()

# Directory for saving samples
os.makedirs("images", exist_ok=True)

def save_generated_images(epoch, generator, fixed_noise):
    generator.eval()
    with torch.no_grad():
        gen_imgs = generator(fixed_noise).reshape(-1, 1, 28, 28)
        gen_imgs = gen_imgs * 0.5 + 0.5  # Denormalize to [0,1]
        grid = torchvision.utils.make_grid(gen_imgs[:25], nrow=5)
        npimg = grid.cpu().numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        plt.title(f'Epoch {epoch}')
        plt.axis('off')
        plt.savefig(f'images/epoch_{epoch}.png')
        plt.close()

# Fixed noise for consistent image generation
fixed_noise = torch.randn(25, latent_dim).to(device)

# Training loop
for epoch in range(1, epochs + 1):
    for i, (imgs, _) in enumerate(train_loader):
        real_imgs = imgs.view(imgs.size(0), -1).to(device)
        real_labels = torch.ones(imgs.size(0), 1).to(device)
        fake_labels = torch.zeros(imgs.size(0), 1).to(device)

        # Train Generator
        optimizer_G.zero_grad()
        z = torch.randn(imgs.size(0), latent_dim).to(device)
        generated_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(generated_imgs), real_labels)
        g_loss.backward()
        optimizer_G.step()

        # Train Discriminator
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs), real_labels)
        fake_loss = adversarial_loss(discriminator(generated_imgs.detach()), fake_labels)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        if i % 200 == 0:
            print(f"Epoch [{epoch}/{epochs}] Batch {i}/{len(train_loader)} \
                  Loss D: {d_loss.item():.4f}, Loss G: {g_loss.item():.4f}")

    # Save sample images
    save_generated_images(epoch, generator, fixed_noise)


100%|██████████| 9.91M/9.91M [00:00<00:00, 16.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 510kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.48MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.95MB/s]


Epoch [1/50] Batch 0/469                   Loss D: 0.7106, Loss G: 0.6589
Epoch [1/50] Batch 200/469                   Loss D: 0.6826, Loss G: 0.3831
Epoch [1/50] Batch 400/469                   Loss D: 0.2527, Loss G: 1.9538
Epoch [2/50] Batch 0/469                   Loss D: 0.2065, Loss G: 1.5228
Epoch [2/50] Batch 200/469                   Loss D: 0.6643, Loss G: 0.3273
Epoch [2/50] Batch 400/469                   Loss D: 0.1327, Loss G: 1.8566
Epoch [3/50] Batch 0/469                   Loss D: 0.5242, Loss G: 4.0002
Epoch [3/50] Batch 200/469                   Loss D: 0.0793, Loss G: 3.9626
Epoch [3/50] Batch 400/469                   Loss D: 0.0467, Loss G: 4.0845
Epoch [4/50] Batch 0/469                   Loss D: 0.2515, Loss G: 1.5327
Epoch [4/50] Batch 200/469                   Loss D: 0.1614, Loss G: 1.7128
Epoch [4/50] Batch 400/469                   Loss D: 0.1650, Loss G: 1.9761
Epoch [5/50] Batch 0/469                   Loss D: 0.1679, Loss G: 3.4195
Epoch [5/50] Batch 200